Currently final model: random forest predict Customers, Lasso predict Sales.

In [ ]:
train = read.csv(file = '../input/457-engineered/train_engineered (2).csv', header = TRUE)
test = read.csv(file = '../input/457-engineered/test_engineered (2).csv', header = TRUE)
train_sub = train[train$Sales > 0, ]#only use positive Sales in training data
testOpen = (test$Open == 1)
test_sub = test[testOpen, ] #only needs to do prediction for open days
colnames(train_sub)

In [ ]:
library(gbm)
library(randomForest)
library(caret)

random forest predict cusotomers

In [ ]:
rfModel = randomForest(Customers ~ ., 
                       data = train_sub[,-c(3,5)], # delete sales and open
                       importance = TRUE, 
                       ntree=1000,
                        mtry = 6)
test_sub$Customers = predict(rfModel, test_sub[,-c(3)])# delete open


In [ ]:
boost.data=gbm(Sales ~ ., 
                 data = train_sub[,-5], # delete open
                 distribution="gaussian", 
                 n.tree=3000,
                 interaction.depth=3,
                 cv.folds = 5)
cv.num = gbm.perf(boost.data)
yhat.boost.cv=predict(boost.data,newdata=test_sub[,-3],n.trees=cv.num)

outDat = data.frame(ID = 1:nrow(test), Sales = rep(0,nrow(test))) 
outDat$Sales[testOpen] = yhat.boost.cv

#Write the prediction to an output file
write.csv(outDat, file = 'Boosting&RandomForest_0316.csv', row.names = FALSE)

In [ ]:
head(outDat)

0.14094

In [ ]:
library(glmnet)

# Random forest + Lasso regression

In [ ]:
dummies <- dummyVars(formula = Sales ~ factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Customers + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):Customers + factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(month) + Customers:Promo + Customers:SchoolHoliday + Customers:factor(month) + Customers:day + Promo:SchoolHoliday + Promo:factor(month) + Promo:day + SchoolHoliday:factor(month) + factor(month):day + CompetitionDistance , data = train_sub)

dummies_test <- dummyVars(formula = ~ factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Customers + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):Customers + factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(month) + Customers:Promo + Customers:SchoolHoliday + Customers:factor(month) + Customers:day + Promo:SchoolHoliday + Promo:factor(month) + Promo:day + SchoolHoliday:factor(month) + factor(month):day + CompetitionDistance , data = test_sub)

train_dummy <-as.matrix(predict(dummies, newdata = train_sub))
test_dummy <-as.matrix(predict(dummies_test, newdata = test_sub))

# make sure dimension of train set is same as test set.
train_dummy <- subset(train_dummy, select = colnames(test_dummy) )

lam.seq = exp(seq(-5, 3, length=100))
lasso.cv.out = cv.glmnet(train_dummy,train_sub$Sales, alpha = 1, standardize = TRUE, lambda = lam.seq) 
lasso.mod <- glmnet(train_dummy, train_sub$Sales,alpha = 1, standardize = TRUE, lambda = lasso.cv.out$lambda.min)

y_pred = predict(lasso.mod, test_dummy)

outDat = data.frame(ID = 1:nrow(test), Sales = rep(0,nrow(test))) 
outDat$Sales[testOpen] = y_pred

#Write the prediction to an output file
write.csv(outDat, file = 'Lasso&RandomForest_0317.csv', row.names = FALSE)

In [ ]:
head(outDat)

0.13096